In [ ]:
# Importing the required Python libraries
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, LSTM
from keras.callbacks import EarlyStopping

import lstm_model_modules as lmm

In [ ]:
training_file_path = 'train_data.csv'
# Extracting only the demand column 
demand_column = lmm.get_demand_column(training_file_path)

In [ ]:
# Splitting the above numpy array into train (80%) and validation sets:
train_size = int(len(demand_column) * 0.8)
train_set = demand_column[:train_size]
val_set = demand_column[train_size:]

In [ ]:
# Calculating the scaler value
scaler = lmm.return_scaler(training_file_path)

# Scaling the train and validation sets
train_scaled = train_set * scaler 
val_scaled = val_set * scaler

In [ ]:
# Preprocessing the train and validation sets, using a specific lookback range
look_back = 336 # since our data is weekly periodic
X_train, Y_train = lmm.create_dataset(train_scaled, look_back)
X_val, Y_val = lmm.create_dataset(val_scaled, look_back)

In [ ]:
# Defining the LSTM model's structure and compiling it
model = Sequential()
model.add(LSTM(units=64, activation='relu', input_shape=(X_train.shape[1], X_train.shape[2]), return_sequences=False))
# model.add(LSTM(units=32))
model.add(Dense(1))
model.compile(optimizer='adam', loss='mse')

# Defining early stopping criteria
es = EarlyStopping(monitor='val_loss', patience=10, verbose=1, mode='min')

In [ ]:
# Fitting the model with early stopping
history = model.fit(X_train, Y_train, epochs=100, batch_size=16, validation_data=(X_val, Y_val), callbacks=[es])

In [ ]:
# Plotting and comparing the training and validation loss
plt.plot(history.history['loss'], label='Training loss')
plt.plot(history.history['val_loss'], label='Validation loss')
plt.legend()

In [ ]:
# Saving the best model
model.save("model_using_only_demand_col.h5")